📦 Cellule 1 : Imports et Configuration


In [4]:
# ========================================
# IMPORTS
# ========================================
import pandas as pd
import pyodbc
from dotenv import load_dotenv
import os
from tqdm import tqdm
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

print("=" * 70)
print("📊 ETL - CHARGEMENT DE FAIT_ENTREPRISE_AUTOMOBILE")
print("=" * 70)

# ========================================
# CHARGEMENT DES VARIABLES D'ENVIRONNEMENT
# ========================================
load_dotenv()

# Configuration DWH
DB_SERVER_DWH = os.getenv('DB_SERVER_DWH-student')
DB_DATABASE_DWH = os.getenv('DB_DATABASE_DWH')
DB_USERNAME_DWH = os.getenv('DB_USERNAME_DWH')
DB_PASSWORD_DWH = os.getenv('DB_PASSWORD_DWH')

print("\n✅ Variables d'environnement chargées")
print(f"   • DWH Server : {DB_SERVER_DWH}")
print(f"   • DWH Database : {DB_DATABASE_DWH}")

📊 ETL - CHARGEMENT DE FAIT_ENTREPRISE_AUTOMOBILE

✅ Variables d'environnement chargées
   • DWH Server : carter-cash-serveur-student.database.windows.net
   • DWH Database : DWH_E5_projet_AUTO


📦 Cellule 2 : Connexion au DWH


In [5]:
print("\n" + "=" * 70)
print("🔗 CONNEXION AU DATA WAREHOUSE")
print("=" * 70)

try:
    driver = '{ODBC Driver 17 for SQL Server}'
    
    connection_string_dwh = (
        'DRIVER=' + driver + 
        ';SERVER=' + DB_SERVER_DWH + 
        ';PORT=1433;DATABASE=' + DB_DATABASE_DWH + 
        ';UID=' + DB_USERNAME_DWH + 
        ';PWD=' + DB_PASSWORD_DWH
    )
    
    cnxn_dwh = pyodbc.connect(connection_string_dwh)
    cursor_dwh = cnxn_dwh.cursor()
    
    print(f"✅ Connexion réussie au DWH")
    print(f"   • Serveur : {DB_SERVER_DWH}")
    print(f"   • Base de données : {DB_DATABASE_DWH}")
    
except Exception as e:
    print(f"❌ Erreur de connexion au DWH : {e}")
    raise


🔗 CONNEXION AU DATA WAREHOUSE
✅ Connexion réussie au DWH
   • Serveur : carter-cash-serveur-student.database.windows.net
   • Base de données : DWH_E5_projet_AUTO


📦 Cellule 3 : Fonction de mapping Tranche_Effectifs → Estimation_Salaries


In [6]:
print("\n" + "=" * 70)
print("🔢 DÉFINITION DU MAPPING TRANCHE_EFFECTIFS → ESTIMATION_SALARIES")
print("=" * 70)

# ========================================
# MAPPING DES TRANCHES D'EFFECTIFS INSEE
# ========================================
# Source : https://www.sirene.fr/sirene/public/variable/tefen

TRANCHE_EFFECTIFS_MAPPING = {
    'NN': None,  # Non renseigné
    '00': 0,     # 0 salarié
    '01': 1,     # 1 ou 2 salariés → moyenne = 1.5 arrondi à 1
    '02': 4,     # 3 à 5 salariés → moyenne = 4
    '03': 7,     # 6 à 9 salariés → moyenne = 7.5 arrondi à 7
    '11': 14,    # 10 à 19 salariés → moyenne = 14.5 arrondi à 14
    '12': 34,    # 20 à 49 salariés → moyenne = 34.5 arrondi à 34
    '21': 74,    # 50 à 99 salariés → moyenne = 74.5 arrondi à 74
    '22': 149,   # 100 à 199 salariés → moyenne = 149.5 arrondi à 149
    '31': 224,   # 200 à 249 salariés → moyenne = 224.5 arrondi à 224
    '32': 374,   # 250 à 499 salariés → moyenne = 374.5 arrondi à 374
    '41': 749,   # 500 à 999 salariés → moyenne = 749.5 arrondi à 749
    '42': 1499,  # 1000 à 1999 salariés → moyenne = 1499.5 arrondi à 1499
    '51': 3499,  # 2000 à 4999 salariés → moyenne = 3499.5 arrondi à 3499
    '52': 7499,  # 5000 à 9999 salariés → moyenne = 7499.5 arrondi à 7499
    '53': 10000, # 10000 salariés et plus → estimation conservatrice = 10000
}

def get_estimation_salaries(tranche):
    """Convertit une tranche d'effectifs en estimation numérique"""
    if pd.isna(tranche) or str(tranche).strip() == '':
        return None
    
    tranche_str = str(tranche).strip().upper()
    return TRANCHE_EFFECTIFS_MAPPING.get(tranche_str, None)

print("✅ Mapping des tranches d'effectifs défini")
print("\n📊 EXEMPLES DE MAPPING :")
for tranche, estimation in list(TRANCHE_EFFECTIFS_MAPPING.items())[:10]:
    if estimation is not None:
        print(f"   • Tranche '{tranche}' → {estimation} salariés (estimation)")
    else:
        print(f"   • Tranche '{tranche}' → Non renseigné")


🔢 DÉFINITION DU MAPPING TRANCHE_EFFECTIFS → ESTIMATION_SALARIES
✅ Mapping des tranches d'effectifs défini

📊 EXEMPLES DE MAPPING :
   • Tranche 'NN' → Non renseigné
   • Tranche '00' → 0 salariés (estimation)
   • Tranche '01' → 1 salariés (estimation)
   • Tranche '02' → 4 salariés (estimation)
   • Tranche '03' → 7 salariés (estimation)
   • Tranche '11' → 14 salariés (estimation)
   • Tranche '12' → 34 salariés (estimation)
   • Tranche '21' → 74 salariés (estimation)
   • Tranche '22' → 149 salariés (estimation)
   • Tranche '31' → 224 salariés (estimation)


📦 Cellule 4 : Extraction des données sources


In [7]:
print("\n" + "=" * 70)
print("📥 EXTRACTION DES DONNÉES SOURCES")
print("=" * 70)

# ========================================
# EXTRAIRE DIM_ENTREPRISE
# ========================================
print("\n⏳ Chargement de DIM_ENTREPRISE...")
query_entreprise = """
SELECT 
    SK_Entreprise,
    SIREN,
    SIRET,
    NAF,
    Code_Commune,
    Tranche_Effectifs,
    Date_Creation,
    Date_Debut_Validite,
    Date_Fin_Validite,
    Flag_Actuel,
    Etat_Administratif
FROM DIM_ENTREPRISE
WHERE Flag_Actuel = 1
  AND Etat_Administratif = 'A'
  AND SIRET IS NOT NULL
  AND NAF IS NOT NULL
  AND Code_Commune IS NOT NULL
"""

df_entreprise = pd.read_sql(query_entreprise, cnxn_dwh)
print(f"✅ {len(df_entreprise)} entreprises actives chargées")

# ========================================
# EXTRAIRE DIM_ACTIVITE_NAF
# ========================================
print("\n⏳ Chargement de DIM_ACTIVITE_NAF...")
query_activite = """
SELECT 
    SK_Activite,
    Code_NAF
FROM DIM_ACTIVITE_NAF
"""

df_activite = pd.read_sql(query_activite, cnxn_dwh)
print(f"✅ {len(df_activite)} codes NAF chargés")

# ========================================
# EXTRAIRE DIM_GEOGRAPHIE
# ========================================
print("\n⏳ Chargement de DIM_GEOGRAPHIE...")
query_geographie = """
SELECT 
    SK_Geographie,
    Code_INSEE
FROM DIM_GEOGRAPHIE
"""

df_geographie = pd.read_sql(query_geographie, cnxn_dwh)
print(f"✅ {len(df_geographie)} communes chargées")

# ========================================
# EXTRAIRE DIM_TEMPS
# ========================================
print("\n⏳ Chargement de DIM_TEMPS...")
query_temps = """
SELECT 
    SK_Temps,
    Date
FROM DIM_TEMPS
"""

df_temps = pd.read_sql(query_temps, cnxn_dwh)
print(f"✅ {len(df_temps)} dates chargées")

print(f"\n📊 APERÇU DES DONNÉES ENTREPRISE :")
print(df_entreprise.head(10))


📥 EXTRACTION DES DONNÉES SOURCES

⏳ Chargement de DIM_ENTREPRISE...
✅ 19028 entreprises actives chargées

⏳ Chargement de DIM_ACTIVITE_NAF...
✅ 18 codes NAF chargés

⏳ Chargement de DIM_GEOGRAPHIE...
✅ 3788 communes chargées

⏳ Chargement de DIM_TEMPS...
✅ 4018 dates chargées

📊 APERÇU DES DONNÉES ENTREPRISE :
   SK_Entreprise     SIREN          SIRET     NAF Code_Commune  \
0              1   7211253   721125300021  4520BC        80001   
1              2   7320161   732016100024  4520AB        80649   
2              3   7320187   732018700029  45.31Z        80001   
3              4   7320187   732018700052  45.31Z        80001   
4              5   7320294   732029400023  45.11Z        80308   
5              6  25480401  2548040101053  45.31Z        59367   
6              7  25480401  2548040101087  45.31Z        60057   
7              8  25480401  2548040101095  45.31Z        59183   
8              9  25480401  2548040101129  45.31Z        59459   
9             10  25480401 

📦 Cellule 5 : Jointures et Préparation des données


In [8]:
print("\n" + "=" * 70)
print("🔗 JOINTURES ET PRÉPARATION DES DONNÉES")
print("=" * 70)

# ========================================
# APPLIQUER LE MAPPING DES EFFECTIFS
# ========================================
print("\n⏳ Application du mapping des effectifs...")
df_entreprise['Estimation_Salaries'] = df_entreprise['Tranche_Effectifs'].apply(get_estimation_salaries)

# Statistiques sur les effectifs
effectifs_renseignes = df_entreprise['Estimation_Salaries'].notna().sum()
effectifs_non_renseignes = df_entreprise['Estimation_Salaries'].isna().sum()

print(f"✅ Mapping appliqué")
print(f"   • Effectifs renseignés : {effectifs_renseignes} ({effectifs_renseignes/len(df_entreprise)*100:.1f}%)")
print(f"   • Effectifs non renseignés : {effectifs_non_renseignes} ({effectifs_non_renseignes/len(df_entreprise)*100:.1f}%)")

# ========================================
# JOINTURE AVEC DIM_ACTIVITE_NAF
# ========================================
print("\n⏳ Jointure avec DIM_ACTIVITE_NAF...")
df_entreprise = df_entreprise.merge(
    df_activite,
    left_on='NAF',
    right_on='Code_NAF',
    how='inner'
)
print(f"✅ {len(df_entreprise)} entreprises après jointure NAF")

# ========================================
# JOINTURE AVEC DIM_GEOGRAPHIE
# ========================================
print("\n⏳ Jointure avec DIM_GEOGRAPHIE...")
df_entreprise = df_entreprise.merge(
    df_geographie,
    left_on='Code_Commune',
    right_on='Code_INSEE',
    how='inner'
)
print(f"✅ {len(df_entreprise)} entreprises après jointure GEOGRAPHIE")

# ========================================
# DÉTERMINER LA DATE DE RÉFÉRENCE
# ========================================
print("\n⏳ Détermination de la date de référence...")

# Utiliser Date_Fin_Validite, sinon Date_Debut_Validite, sinon Date_Creation
def get_date_reference(row):
    if pd.notna(row['Date_Fin_Validite']):
        return row['Date_Fin_Validite']
    elif pd.notna(row['Date_Debut_Validite']):
        return row['Date_Debut_Validite']
    elif pd.notna(row['Date_Creation']):
        return row['Date_Creation']
    else:
        return None

df_entreprise['Date_Reference'] = df_entreprise.apply(get_date_reference, axis=1)

# Supprimer les lignes sans date de référence
nb_avant = len(df_entreprise)
df_entreprise = df_entreprise[df_entreprise['Date_Reference'].notna()]
nb_apres = len(df_entreprise)

if nb_avant != nb_apres:
    print(f"⚠️  {nb_avant - nb_apres} entreprises supprimées (pas de date de référence)")

print(f"✅ {len(df_entreprise)} entreprises avec date de référence")

# ========================================
# JOINTURE AVEC DIM_TEMPS
# ========================================
print("\n⏳ Jointure avec DIM_TEMPS...")

# Convertir Date_Reference en date seule (sans heure)
df_entreprise['Date_Reference'] = pd.to_datetime(df_entreprise['Date_Reference']).dt.date

# Convertir la colonne Date de df_temps en date
df_temps['Date'] = pd.to_datetime(df_temps['Date']).dt.date

df_entreprise = df_entreprise.merge(
    df_temps,
    left_on='Date_Reference',
    right_on='Date',
    how='inner'
)
print(f"✅ {len(df_entreprise)} entreprises après jointure TEMPS")

# ========================================
# SÉLECTIONNER LES COLONNES NÉCESSAIRES
# ========================================
print("\n⏳ Sélection des colonnes pour la table de faits...")

df_fait = df_entreprise[[
    'SK_Temps',
    'SK_Entreprise',
    'SK_Activite',
    'SK_Geographie',
    'SIRET',
    'Estimation_Salaries'
]].copy()

print(f"✅ {len(df_fait)} lignes préparées")

print(f"\n📊 APERÇU DES DONNÉES PRÉPARÉES :")
print(df_fait.head(10))


🔗 JOINTURES ET PRÉPARATION DES DONNÉES

⏳ Application du mapping des effectifs...
✅ Mapping appliqué
   • Effectifs renseignés : 4495 (23.6%)
   • Effectifs non renseignés : 14533 (76.4%)

⏳ Jointure avec DIM_ACTIVITE_NAF...
✅ 13493 entreprises après jointure NAF

⏳ Jointure avec DIM_GEOGRAPHIE...
✅ 12398 entreprises après jointure GEOGRAPHIE

⏳ Détermination de la date de référence...
✅ 12398 entreprises avec date de référence

⏳ Jointure avec DIM_TEMPS...
✅ 12398 entreprises après jointure TEMPS

⏳ Sélection des colonnes pour la table de faits...
✅ 12398 lignes préparées

📊 APERÇU DES DONNÉES PRÉPARÉES :
   SK_Temps  SK_Entreprise  SK_Activite  SK_Geographie          SIRET  \
0      2167              3           13           3024   732018700029   
1      2166              4           13           3024   732018700052   
2      2166              5            4           3309   732029400023   
3      2167              6           13           1155  2548040101053   
4      2166         

📦 Cellule 6 : Agrégation et Calcul de Nombre_Etablissements


In [9]:
print("\n" + "=" * 70)
print("📊 AGRÉGATION ET CALCUL DES MÉTRIQUES")
print("=" * 70)

# ========================================
# AGRÉGATION PAR DIMENSIONS
# ========================================
print("\n⏳ Agrégation des données par dimensions...")

# Grouper par les 4 dimensions et calculer les métriques
df_fait_agg = df_fait.groupby([
    'SK_Temps',
    'SK_Entreprise',
    'SK_Activite',
    'SK_Geographie'
]).agg({
    'SIRET': 'count',  # Nombre d'établissements (normalement 1 par ligne)
    'Estimation_Salaries': 'first'  # Prendre la première valeur (devrait être la même)
}).reset_index()

# Renommer les colonnes
df_fait_agg = df_fait_agg.rename(columns={
    'SIRET': 'Nombre_Etablissements'
})

print(f"✅ Agrégation terminée : {len(df_fait_agg)} lignes")

# ========================================
# GÉRER LES VALEURS NULL POUR ESTIMATION_SALARIES
# ========================================
print("\n⏳ Gestion des valeurs NULL pour Estimation_Salaries...")

# Les valeurs NULL dans Estimation_Salaries doivent être insérées comme NULL en SQL
# On va les garder comme NaN pour l'instant

null_count = df_fait_agg['Estimation_Salaries'].isna().sum()
print(f"   • Lignes avec Estimation_Salaries NULL : {null_count}")

# ========================================
# STATISTIQUES SUR LES DONNÉES AGRÉGÉES
# ========================================
print("\n📊 STATISTIQUES SUR LES DONNÉES AGRÉGÉES :")
print(f"   • Total de lignes : {len(df_fait_agg)}")
print(f"   • Total d'établissements : {df_fait_agg['Nombre_Etablissements'].sum()}")
print(f"   • Estimation totale de salariés : {df_fait_agg['Estimation_Salaries'].sum():.0f}" if df_fait_agg['Estimation_Salaries'].notna().any() else "   • Estimation totale de salariés : Non calculable")
print(f"   • Nombre de SK_Temps uniques : {df_fait_agg['SK_Temps'].nunique()}")
print(f"   • Nombre d'entreprises uniques : {df_fait_agg['SK_Entreprise'].nunique()}")
print(f"   • Nombre d'activités NAF uniques : {df_fait_agg['SK_Activite'].nunique()}")
print(f"   • Nombre de communes uniques : {df_fait_agg['SK_Geographie'].nunique()}")

print(f"\n📊 APERÇU DES DONNÉES AGRÉGÉES :")
print(df_fait_agg.head(20))


📊 AGRÉGATION ET CALCUL DES MÉTRIQUES

⏳ Agrégation des données par dimensions...
✅ Agrégation terminée : 12398 lignes

⏳ Gestion des valeurs NULL pour Estimation_Salaries...
   • Lignes avec Estimation_Salaries NULL : 10566

📊 STATISTIQUES SUR LES DONNÉES AGRÉGÉES :
   • Total de lignes : 12398
   • Total d'établissements : 12398
   • Estimation totale de salariés : 21792
   • Nombre de SK_Temps uniques : 105
   • Nombre d'entreprises uniques : 12398
   • Nombre d'activités NAF uniques : 7
   • Nombre de communes uniques : 1641

📊 APERÇU DES DONNÉES AGRÉGÉES :
    SK_Temps  SK_Entreprise  SK_Activite  SK_Geographie  \
0       2067           9194            9           1876   
1       2067          18332            4           1384   
2       2067          18344            9           1013   
3       2067          18365            9           2563   
4       2068           1635            9           2016   
5       2068           5330            4           2637   
6       2068       

📦 Cellule 7 : Chargement dans FAIT_ENTREPRISE_AUTOMOBILE


In [10]:
print("\n" + "=" * 70)
print("📥 ETL - CHARGEMENT DE FAIT_ENTREPRISE_AUTOMOBILE")
print("=" * 70)

# ========================================
# VÉRIFIER LES ENREGISTREMENTS EXISTANTS
# ========================================
print("\n⏳ Vérification des enregistrements existants...")

cursor_dwh.execute("""
SELECT COUNT(*) 
FROM FAIT_ENTREPRISE_AUTOMOBILE
""")
nb_existants = cursor_dwh.fetchone()[0]
print(f"✅ {nb_existants} enregistrements déjà présents dans FAIT_ENTREPRISE_AUTOMOBILE")

# ========================================
# FILTRER LES NOUVEAUX ENREGISTREMENTS
# ========================================
print("\n⏳ Filtrage des nouveaux enregistrements...")

# Charger les combinaisons existantes
query_existants = """
SELECT 
    SK_Temps,
    SK_Entreprise,
    SK_Activite,
    SK_Geographie
FROM FAIT_ENTREPRISE_AUTOMOBILE
"""

if nb_existants > 0:
    df_existants = pd.read_sql(query_existants, cnxn_dwh)
    
    # Créer une colonne de clé composite pour faciliter la comparaison
    df_existants['cle_composite'] = (
        df_existants['SK_Temps'].astype(str) + '_' +
        df_existants['SK_Entreprise'].astype(str) + '_' +
        df_existants['SK_Activite'].astype(str) + '_' +
        df_existants['SK_Geographie'].astype(str)
    )
    
    df_fait_agg['cle_composite'] = (
        df_fait_agg['SK_Temps'].astype(str) + '_' +
        df_fait_agg['SK_Entreprise'].astype(str) + '_' +
        df_fait_agg['SK_Activite'].astype(str) + '_' +
        df_fait_agg['SK_Geographie'].astype(str)
    )
    
    # Filtrer les nouveaux
    df_fait_nouveaux = df_fait_agg[~df_fait_agg['cle_composite'].isin(df_existants['cle_composite'])]
    
    # Supprimer la colonne temporaire
    df_fait_nouveaux = df_fait_nouveaux.drop(columns=['cle_composite'])
    
    skip_count = len(df_fait_agg) - len(df_fait_nouveaux)
    
    print(f"✅ {len(df_fait_nouveaux)} nouveaux enregistrements à insérer")
    print(f"⚠️  {skip_count} enregistrements déjà existants (ignorés)")
else:
    df_fait_nouveaux = df_fait_agg.copy()
    skip_count = 0
    print(f"✅ {len(df_fait_nouveaux)} enregistrements à insérer (table vide)")

# ========================================
# INSERTION PAR BATCH
# ========================================
if len(df_fait_nouveaux) > 0:
    print("\n⏳ Insertion des nouveaux enregistrements...")
    
    insert_query = """
    INSERT INTO FAIT_ENTREPRISE_AUTOMOBILE (
        SK_Temps,
        SK_Entreprise,
        SK_Activite,
        SK_Geographie,
        Nombre_Etablissements,
        Estimation_Salaries
    ) VALUES (?, ?, ?, ?, ?, ?)
    """
    
    batch_size = 1000
    total_insert = 0
    
    # Préparer les tuples pour l'insertion
    tuples = []
    for _, row in df_fait_nouveaux.iterrows():
        tuples.append((
            int(row['SK_Temps']),
            int(row['SK_Entreprise']),
            int(row['SK_Activite']),
            int(row['SK_Geographie']),
            int(row['Nombre_Etablissements']),
            int(row['Estimation_Salaries']) if pd.notna(row['Estimation_Salaries']) else None
        ))
    
    # Insertion par batch
    for i in tqdm(range(0, len(tuples), batch_size), desc="Insertion faits", unit="batch"):
        batch = tuples[i:i+batch_size]
        try:
            cursor_dwh.executemany(insert_query, batch)
            cnxn_dwh.commit()
            total_insert += len(batch)
        except Exception as e:
            print(f"\n⚠️  Erreur batch {i//batch_size+1}: {e}")
            # Insérer ligne par ligne en cas d'erreur
            for row_data in batch:
                try:
                    cursor_dwh.execute(insert_query, row_data)
                    cnxn_dwh.commit()
                    total_insert += 1
                except Exception as e_detail:
                    print(f"   ❌ Erreur : {e_detail}")
                    print(f"   Données : {row_data}")
    
    print(f"\n✅ Insertion terminée : {total_insert} enregistrements insérés")
else:
    print("\n✅ Aucun nouvel enregistrement à insérer")
    total_insert = 0


📥 ETL - CHARGEMENT DE FAIT_ENTREPRISE_AUTOMOBILE

⏳ Vérification des enregistrements existants...
✅ 0 enregistrements déjà présents dans FAIT_ENTREPRISE_AUTOMOBILE

⏳ Filtrage des nouveaux enregistrements...
✅ 12398 enregistrements à insérer (table vide)

⏳ Insertion des nouveaux enregistrements...


Insertion faits: 100%|██████████| 13/13 [02:02<00:00,  9.39s/batch]


✅ Insertion terminée : 12398 enregistrements insérés


📦 Cellule 8 : Vérifications finales


In [11]:
print("\n" + "=" * 70)
print("🔍 VÉRIFICATION FINALE")
print("=" * 70)

# ========================================
# STATISTIQUES GLOBALES
# ========================================
cursor_dwh.execute("SELECT COUNT(*) FROM FAIT_ENTREPRISE_AUTOMOBILE")
total_final = cursor_dwh.fetchone()[0]
print(f"\n📊 Nombre total d'enregistrements dans FAIT_ENTREPRISE_AUTOMOBILE : {total_final}")

# Total des établissements
cursor_dwh.execute("""
SELECT 
    SUM(Nombre_Etablissements) as Total_Etablissements,
    SUM(Estimation_Salaries) as Total_Salaries_Estimation,
    COUNT(CASE WHEN Estimation_Salaries IS NULL THEN 1 END) as Nb_Sans_Estimation
FROM FAIT_ENTREPRISE_AUTOMOBILE
""")

row = cursor_dwh.fetchone()
print(f"\n📈 MÉTRIQUES GLOBALES :")
print(f"   • Total d'établissements : {row[0]}")
print(f"   • Estimation totale de salariés : {int(row[1]) if row[1] else 'Non calculable'}")
print(f"   • Enregistrements sans estimation : {row[2]}")

# Répartition par activité NAF
print("\n📈 TOP 10 DES ACTIVITÉS NAF :")
cursor_dwh.execute("""
SELECT TOP 10
    a.Code_NAF,
    a.Intitule_NAF_40,
    SUM(f.Nombre_Etablissements) as Total_Etablissements,
    SUM(f.Estimation_Salaries) as Total_Salaries
FROM FAIT_ENTREPRISE_AUTOMOBILE f
INNER JOIN DIM_ACTIVITE_NAF a ON f.SK_Activite = a.SK_Activite
GROUP BY a.Code_NAF, a.Intitule_NAF_40
ORDER BY Total_Etablissements DESC
""")

for row in cursor_dwh.fetchall():
    code_naf = row[0]
    intitule = row[1]
    nb_etab = row[2]
    nb_salaries = int(row[3]) if row[3] else 0
    print(f"   • {code_naf} - {intitule}")
    print(f"     └─ {nb_etab} établissements | {nb_salaries} salariés (estimation)")

# Répartition par commune (TOP 10)
print("\n📈 TOP 10 DES COMMUNES :")
cursor_dwh.execute("""
SELECT TOP 10
    g.Nom_Commune,
    g.Code_INSEE,
    g.Code_Postal,
    SUM(f.Nombre_Etablissements) as Total_Etablissements,
    SUM(f.Estimation_Salaries) as Total_Salaries
FROM FAIT_ENTREPRISE_AUTOMOBILE f
INNER JOIN DIM_GEOGRAPHIE g ON f.SK_Geographie = g.SK_Geographie
GROUP BY g.Nom_Commune, g.Code_INSEE, g.Code_Postal
ORDER BY Total_Etablissements DESC
""")

for row in cursor_dwh.fetchall():
    commune = row[0]
    code_insee = row[1]
    code_postal = row[2]
    nb_etab = row[3]
    nb_salaries = int(row[4]) if row[4] else 0
    print(f"   • {commune} ({code_postal})")
    print(f"     └─ {nb_etab} établissements | {nb_salaries} salariés (estimation)")

# Répartition par année
print("\n📈 RÉPARTITION PAR ANNÉE :")
cursor_dwh.execute("""
SELECT 
    t.Annee,
    SUM(f.Nombre_Etablissements) as Total_Etablissements,
    SUM(f.Estimation_Salaries) as Total_Salaries
FROM FAIT_ENTREPRISE_AUTOMOBILE f
INNER JOIN DIM_TEMPS t ON f.SK_Temps = t.SK_Temps
GROUP BY t.Annee
ORDER BY t.Annee
""")

for row in cursor_dwh.fetchall():
    annee = row[0]
    nb_etab = row[1]
    nb_salaries = int(row[2]) if row[2] else 0
    print(f"   • {annee} : {nb_etab} établissements | {nb_salaries} salariés (estimation)")

# ========================================
# EXEMPLES D'ENREGISTREMENTS
# ========================================
print("\n📋 EXEMPLES D'ENREGISTREMENTS DANS LA TABLE :")
print("=" * 70)
cursor_dwh.execute("""
SELECT TOP 10
    t.Date,
    e.SIRET,
    e.Denomination,
    a.Code_NAF,
    a.Intitule_NAF_40,
    g.Nom_Commune,
    g.Code_Postal,
    f.Nombre_Etablissements,
    f.Estimation_Salaries
FROM FAIT_ENTREPRISE_AUTOMOBILE f
INNER JOIN DIM_TEMPS t ON f.SK_Temps = t.SK_Temps
INNER JOIN DIM_ENTREPRISE e ON f.SK_Entreprise = e.SK_Entreprise
INNER JOIN DIM_ACTIVITE_NAF a ON f.SK_Activite = a.SK_Activite
INNER JOIN DIM_GEOGRAPHIE g ON f.SK_Geographie = g.SK_Geographie
ORDER BY t.Date DESC
""")

print(f"\n{'Date':<12} {'SIRET':<15} {'Dénomination':<25} {'NAF':<8} {'Commune':<20} {'Nb Etab':<8} {'Salariés':<10}")
print("-" * 130)

for row in cursor_dwh.fetchall():
    date_str = row[0].strftime('%Y-%m-%d') if row[0] else ''
    siret = row[1] if row[1] else ''
    denom = row[2][:25] if row[2] else ''
    naf = row[3] if row[3] else ''
    commune = row[5][:20] if row[5] else ''
    nb_etab = row[7]
    salaries = int(row[8]) if row[8] else 0
    
    print(f"{date_str:<12} {siret:<15} {denom:<25} {naf:<8} {commune:<20} {nb_etab:<8} {salaries:<10}")

# ========================================
# VÉRIFICATION DE L'INTÉGRITÉ RÉFÉRENTIELLE
# ========================================
print("\n🔍 VÉRIFICATION DE L'INTÉGRITÉ RÉFÉRENTIELLE :")
print("=" * 70)

# Vérifier les FK vers DIM_TEMPS
cursor_dwh.execute("""
SELECT COUNT(*) 
FROM FAIT_ENTREPRISE_AUTOMOBILE f
LEFT JOIN DIM_TEMPS t ON f.SK_Temps = t.SK_Temps
WHERE t.SK_Temps IS NULL
""")
nb_orphelins_temps = cursor_dwh.fetchone()[0]

# Vérifier les FK vers DIM_ENTREPRISE
cursor_dwh.execute("""
SELECT COUNT(*) 
FROM FAIT_ENTREPRISE_AUTOMOBILE f
LEFT JOIN DIM_ENTREPRISE e ON f.SK_Entreprise = e.SK_Entreprise
WHERE e.SK_Entreprise IS NULL
""")
nb_orphelins_entreprise = cursor_dwh.fetchone()[0]

# Vérifier les FK vers DIM_ACTIVITE_NAF
cursor_dwh.execute("""
SELECT COUNT(*) 
FROM FAIT_ENTREPRISE_AUTOMOBILE f
LEFT JOIN DIM_ACTIVITE_NAF a ON f.SK_Activite = a.SK_Activite
WHERE a.SK_Activite IS NULL
""")
nb_orphelins_activite = cursor_dwh.fetchone()[0]

# Vérifier les FK vers DIM_GEOGRAPHIE
cursor_dwh.execute("""
SELECT COUNT(*) 
FROM FAIT_ENTREPRISE_AUTOMOBILE f
LEFT JOIN DIM_GEOGRAPHIE g ON f.SK_Geographie = g.SK_Geographie
WHERE g.SK_Geographie IS NULL
""")
nb_orphelins_geographie = cursor_dwh.fetchone()[0]

print(f"   • Enregistrements orphelins DIM_TEMPS : {nb_orphelins_temps}")
print(f"   • Enregistrements orphelins DIM_ENTREPRISE : {nb_orphelins_entreprise}")
print(f"   • Enregistrements orphelins DIM_ACTIVITE_NAF : {nb_orphelins_activite}")
print(f"   • Enregistrements orphelins DIM_GEOGRAPHIE : {nb_orphelins_geographie}")

if (nb_orphelins_temps + nb_orphelins_entreprise + nb_orphelins_activite + nb_orphelins_geographie) == 0:
    print("\n✅ Intégrité référentielle OK : Aucun enregistrement orphelin")
else:
    print("\n⚠️  ATTENTION : Des enregistrements orphelins ont été détectés !")

# ========================================
# RÉSUMÉ FINAL
# ========================================
print("\n" + "=" * 70)
print("✅ CHARGEMENT TERMINÉ - FAIT_ENTREPRISE_AUTOMOBILE")
print("=" * 70)
print(f"   📊 Total d'enregistrements dans la table : {total_final}")
print(f"   ➕ Nouveaux enregistrements insérés : {total_insert}")
print(f"   ⏭️  Enregistrements déjà existants : {skip_count}")
print("=" * 70)


🔍 VÉRIFICATION FINALE

📊 Nombre total d'enregistrements dans FAIT_ENTREPRISE_AUTOMOBILE : 12398

📈 MÉTRIQUES GLOBALES :
   • Total d'établissements : 12398
   • Estimation totale de salariés : 21792
   • Enregistrements sans estimation : 10566

📈 TOP 10 DES ACTIVITÉS NAF :
   • 45.11Z - Comm. de voiture & véhicule auto. léger
     └─ 6352 établissements | 11447 salariés (estimation)
   • 45.20A - Entretien & répar. véhicule auto. léger
     └─ 4041 établissements | 2388 salariés (estimation)
   • 45.32Z - Commerce de détail équipement automobile
     └─ 999 établissements | 3384 salariés (estimation)
   • 45.31Z - Commerce de gros d'équipement automobile
     └─ 473 établissements | 2924 salariés (estimation)
   • 45.40Z - Commerce et réparation de motocycles
     └─ 243 établissements | 157 salariés (estimation)
   • 45.19Z - Commerce d'autres véhicules automobiles
     └─ 161 établissements | 720 salariés (estimation)
   • 45.20B - Entretien & répar. autre véhicule auto.
     └─ 129

📦 Cellule 9 : Fermeture des connexions


In [12]:
print("\n" + "=" * 70)
print("🔒 FERMETURE DES CONNEXIONS")
print("=" * 70)

# ========================================
# FERMETURE DU CURSEUR DWH
# ========================================
try:
    if 'cursor_dwh' in locals() and cursor_dwh:
        cursor_dwh.close()
        print("✅ Curseur DWH fermé")
except Exception as e:
    print(f"⚠️  Erreur lors de la fermeture du curseur : {e}")

# ========================================
# FERMETURE DE LA CONNEXION DWH
# ========================================
try:
    if 'cnxn_dwh' in locals() and cnxn_dwh:
        cnxn_dwh.close()
        print("✅ Connexion DWH fermée")
except Exception as e:
    print(f"⚠️  Erreur lors de la fermeture de la connexion : {e}")

print("\n" + "=" * 70)
print("✅ Toutes les connexions ont été fermées avec succès")
print("=" * 70)
print("\n🎉 FIN DU NOTEBOOK - ETL FAIT_ENTREPRISE_AUTOMOBILE TERMINÉ")
print("\n📊 RÉSUMÉ DES OPÉRATIONS :")
print(f"   • Table de faits : FAIT_ENTREPRISE_AUTOMOBILE")
print(f"   • Dimensions utilisées : DIM_TEMPS, DIM_ENTREPRISE, DIM_ACTIVITE_NAF, DIM_GEOGRAPHIE")
print(f"   • Métrique calculée : Nombre_Etablissements (COUNT DISTINCT)")
print(f"   • Métrique estimée : Estimation_Salaries (mapping depuis Tranche_Effectifs)")
print(f"   • Total d'enregistrements : {total_final}")
print(f"   • Nouveaux enregistrements : {total_insert}")
print("\n💾 Votre table de faits FAIT_ENTREPRISE_AUTOMOBILE est maintenant à jour !")
print("=" * 70)


🔒 FERMETURE DES CONNEXIONS
✅ Curseur DWH fermé
✅ Connexion DWH fermée

✅ Toutes les connexions ont été fermées avec succès

🎉 FIN DU NOTEBOOK - ETL FAIT_ENTREPRISE_AUTOMOBILE TERMINÉ

📊 RÉSUMÉ DES OPÉRATIONS :
   • Table de faits : FAIT_ENTREPRISE_AUTOMOBILE
   • Dimensions utilisées : DIM_TEMPS, DIM_ENTREPRISE, DIM_ACTIVITE_NAF, DIM_GEOGRAPHIE
   • Métrique calculée : Nombre_Etablissements (COUNT DISTINCT)
   • Métrique estimée : Estimation_Salaries (mapping depuis Tranche_Effectifs)
   • Total d'enregistrements : 12398
   • Nouveaux enregistrements : 12398

💾 Votre table de faits FAIT_ENTREPRISE_AUTOMOBILE est maintenant à jour !
